## Features Comparison Notebook
---

### Objective:
- To assess the impact of different features on the performance of the model.
- To determine the optimal features for the model.

### Methodology:
- A model (RF, SVM, LR) is trained with different features (Chroma, MFCC_30, MFCC_120, CQT_30, CQT_70). The features are used 'as extracted' without any additional processing.
- The model is trained with 80% of the data and tested with the remaining 20%.
  
#### Findings:
- 